In [ ]:
%%shell
gdown 1SP1Z5shLhlSel8xA42bIJ0wwEj3zfxml
gdown 1a-ndlzx5bLsIqNcjK0mVPjl1B3jRcGus
gdown 1gpAKzdBKSlv40AabkLNjLX0h01G9TMzH

Downloading...
From: https://drive.google.com/uc?id=1SP1Z5shLhlSel8xA42bIJ0wwEj3zfxml
To: /content/task.md
100% 5.72k/5.72k [00:00<00:00, 26.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a-ndlzx5bLsIqNcjK0mVPjl1B3jRcGus
To: /content/test.json
100% 1.35M/1.35M [00:00<00:00, 170MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gpAKzdBKSlv40AabkLNjLX0h01G9TMzH
To: /content/train.json
100% 8.21M/8.21M [00:00<00:00, 91.7MB/s]


In [ ]:
from IPython.display import display, Markdown

with open('task.md') as file_:
    display(Markdown(file_.read()))

# Введение
В Контуре мы много работаем с документами: арбитражные иски, госзакупки, исполнительные производства. В данном задании мы предлагаем вам сделать модель, которая поможет отделу госзакупок извлекать 
нужный кусок текста из документа для того, чтобы сформировать анкету заявки. То, какой именно фрагмент текста нужно извлечь, зависит от пункта анкеты, соответствующего документу.
Всего в каждом документе, с которыми вы будет работать, есть 1 из 2-х пунктов анкеты, по которым необходимо извлекать кусочки из текста:
- обеспечение исполнения контракта
- обеспечение гарантийных обязательств

Соответственно, ваша модель, принимая на вход `текст документа` и `наименование одного из двух пунктов`, должна возвращать `соответствующий кусочек текста из текста документа`.

# Данные

### train.json 
Данные для обучения в формате json имеют следующие поля:
- `id`: int - id документа
-  `text`: str - текст документа, в котором может содержаться фрагмент текста, соответствующий пункту анкеты из поля `label`
- `label`: str - название пункта анкеты. Может принимать одно из двух значений: `обеспечение исполнения контракта` или `обеспечение гарантийных обязательств`
- `extracted_part`: dict следующего формата:
    ```
    {
        'text': [фрагмент текста из поля `text`, соответствующий пункту анкеты], 
        'answer_start': [индекс символа начала фрагмента текста в тексте документа],
        'answer_end': [индекс символа конца фрагмента текста в тексте документа]
    }
   ```
  
### test.json

Для демонстрации работы модели используйте данные из файла `test.json`. В нем есть все те же поля, что и в файле `train.json`, кроме поля `extracted_part` - именно его вам и нужно будет добавить,
для того, чтобы мы смогли оценить качество вашей модели.

# Тестовое задание

Для выполнения тестового задания требуется разработать модель, которая будет способна по паре `текст документа` и `пункт анкеты` извлекать из текста документа нужный фрагмент текста. 
Обучив модель, добавьте в файл `test.json` поле `extracted_part` в том же формате, что и в файле `train.json`. Новый файл назовите `predictions.json`

**Подсказка**: изучив данные, вы можете заметить, что у части наблюдений отсутствует фрагмент текста к извлечению (пустая строка внутри поля `extracted_part` с `answer_start` и
`answer_end` равными нулю). Это означает, что в тексте документа нет нужного фрагмента текста, соответствующего пункту анкеты. Учтите это в обучении вашей модели и при формировании
файла с ответами.

# Критерии оценки
1. Для оценки финального решения будет использоваться метрика `Accuracy`: доля наблюдений, в которых извлеченный моделью фрагмент текста полностью соответствует фактически
   требуемому фрагменту.
2. Чистота кода, оформление и понятность исследования.

# Требования к решению
В качестве решения мы ожидаем zip-архив со всеми *.py и *.ipynb файлами в папке solution и файлом `predictions.json` в корне. Формат имени zip-архива: LastName_FirstName.zip (пример Ivanov_Ivan.zip).
Файл `predictions.json` должен включать в себя колонки `id`, `text`, `label`, содержащие те же данные, что и исходный файл `test.json`, а также колонку `extracted_part` в том же
формате, что и в файле `train.json`
Разметка тестового набора данных и включение его в обучение/валидацию запрещены.

В папке solution должно быть отражено исследование и весь код, необходимый для воспроизведения исследования.

Успехов!

In [ ]:
import json

with open('train.json') as file_:
    train = json.loads(file_.read())

with open('test.json') as file_:
    test = json.loads(file_.read())

In [ ]:
print(f'Size of the train subset: {len(train)}')
print(f'Size of the test subset: {len(test)}')

Size of the train subset: 1799
Size of the test subset: 318


In [ ]:
print(f'Original train subset proportion: '
      f'{len(train) / (len(train) + len(test)):.2f}')

Original train subset proportion: 0.85


In [ ]:
import numpy as np

extracted_part_lens = [len(sample['extracted_part']['text'][0])
                       for sample in train]

print(f'Average length of extracted part: '
      f'{np.asarray(extracted_part_lens).mean():.2f}')

Average length of extracted part: 87.89


In [ ]:
for sample in train:
    if '_' in  sample['extracted_part']['text'][0]:
        from pprint import pprint
        
        pprint(sample)
        # print(sample['extracted_part']['text'][0])

Streaming output truncated to the last 5000 lines.
         '9.7. В ходе исполнения Контракта Исполнитель вправе изменить способ '
         'обеспечения исполнения Контракта и (или) предоставить Заказчику '
         'взамен ранее предоставленного обеспечения исполнения Контракта новое '
         'обеспечение исполнения Контракта, размер которого может быть '
         'уменьшен в порядке и случаях, которые предусмотрены частями 7.2 и '
         '7.3 статьи 96 Федерального закона от 05.04.2013 № 44-ФЗ. 9.8. - '
         'адреса домов на территории муниципального района Бирский район '
         'Республики Башкортостан, в которых не приобретаются жилые помещения '
         'детям-сиротам и детям, оставшимся без попечения родителей, лицам из '
         'числа детей-сирот и детей,'}
{'extracted_part': {'answer_end': [1412],
                    'answer_start': [1269],
                    'text': ['Обеспечение исполнения настоящего Контракта '
                             'устанавливается в р

In [ ]:
from pprint import pprint
pprint(train[307])

{'extracted_part': {'answer_end': [1314],
                    'answer_start': [1193],
                    'text': ['Обеспечение исполнения Контракта устанавливается '
                             'в размере 5% от цены Контракта и составляет '
                             '_____________________ (_____']},
 'id': 11679309,
 'label': 'обеспечение исполнения контракта',
 'text': 'ПРОЕКТ Государственный контракт №________ на поставку источников '
         'бесперебойного питания (Идентификационный код закупки № '
         '222450121796445010100100460022790242) г. Курган '
         '«____»__________20___г. онтракта не превышает 3 млн. рублей; 5 '
         'процентов цены Контракта в случае, если цена Контракта составляет от '
         '3 млн. рублей до 50 млн. рублей (включительно); 1 процент цены '
         'Контракта в случае, если цена Контракта составляет от 50 млн. рублей '
         'до 100 млн. рублей (включительно). 6.4. Общая сумма начисленных '
         'штрафов за неисполнение или 

In [ ]:
i = 0
for sample in train:
    if '_' in  sample['extracted_part']['text'][0]:
        print(i, sample['extracted_part']['text'][0])

    i += 1

5 Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет ___
23 Размер обеспечения исполнения контракта составляет 10 (Десять) процентов цены контракта, что составляет _____ рублей.
26 Исполнитель внес обеспечение исполнения Контракта в размере ___ , что составляет 5 % Цены Контракта в форме безотзывной независимой гарантии
39 Обеспечение исполнения настоящего Контракта установлено в размере 5,00% от цены контракта, а именно _______ рублей ____копеек.
42 Обеспечение исполнения Контракта устанавливается в размере 5 % от цены, по которой заключается Контракт, что составляет __
45 Обеспечение исполнения Контракта устанавливается в размере 0,5 % от цены Контракта, что составляет ___________ (____________
55 Принять к сведению, что Поставщик внес обеспечение исполнения контракта в размере 10,00% от цены контракта на сумму _________ рублей.
58 Обеспечение исполнения контракта установлено в размере 5,00% от це

In [ ]:
for sample in train:
    if '_' in  sample['extracted_part']['text'][0] and 'пред' == sample['extracted_part']['text'][0][:4]:
        print(sample)

{'id': 750390013, 'text': 'Приложение № 3 к извещению о проведении электронного аукциона ИКЗ № 222 0278176470 027601001 0362 002 2822 414 ГОСУДАРСТВЕННЫЙ КОНТРАКТ № 06/2022-_______ г. Уфа « ___ » ___________ 2022 г. чи заявки на участие в закупке трех контрактов, исполненных без применения к нему неустоек (штрафов, пеней). Такая информация представляется Поставщиком до заключения Контракта. При этом сумма цен таких контрактов должна составлять не менее начальной (максимальной) цены Контракта, указанной в извещении об осуществлении закупки и документации о закупке. 5.2. Поставщик до заключения Контракта предоставляет Государственному заказчику обеспечение исполнения Контракта в размере 20 % цены Контракта, что составляет ________________ (___________________) рублей. В случае, предусмотренным абзацем 3 п. 5.1. Контракта, обеспечение исполнения Контракта Поставщиком не предоставляется. В случае если предложенная Поставщиком цена Контракта снижена на 25% и более по отношению к начальной (

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 2\. Dataset Preprocessing
- Find mean between target sentence embeddings
- Truncate sentences from the beginning and the end of samples

### 2\.0 Dependencies

In [ ]:
!python -qq -m spacy download ru_core_news_lg
!pip -qq install natasha

2023-05-07 22:15:26.792629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 22:15:27.922031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=00ef968e0f3bded3add5195e2cdbf1851ff05ec169fef2dc6119829132c74e40
  Stored in directory: /root/.

In [ ]:
import spacy
from natasha import Doc, Segmenter, NewsEmbedding

import typing
from tqdm.notebook import tqdm
from pprint import pprint

spacy_model = spacy.load("ru_core_news_lg")
natasha_segmenter = Segmenter()
natasha_emb = NewsEmbedding()

EMB_DIM = 300

### 2\.1 Punctuation Restoration

In [ ]:
PUNCT_RESTORE_MODE = None

#### 2\.1\.1 Using Regex

In [ ]:
import re
import copy
import typing

In [ ]:
def restore_punct_with_regex(data: typing.Dict) -> typing.Dict:
    data_copy = copy.deepcopy(data)

    for i in range(len(data_copy)):
        data_copy[i]['text'] = re.sub(r'(\s+[А-ЯЁ][а-яё]+)', r'.\1',
                                      data_copy[i]['text'])

    return data_copy

In [ ]:
if PUNCT_RESTORE_MODE == 'REGEX':
    temp = restore_punct_with_regex(train)

    for old, new in zip(train, temp):
        old_model = spacy_model(old['text'])
        new_model = spacy_model(new['text'])

        old_len = len(list(old_model.sents))
        new_len = len(list(new_model.sents))

        if old_len != new_len:
            print()
            pprint(list(old_model.sents))

            print()
            pprint(list(new_model.sents))

    train = temp

### 2\.2 Text Core Extraction

In [ ]:
SEGMENTATION_MODEL = 'CUSTOM'
SIMILARITY_THRESHOLD = 0.78

#### 2\.2\.1 Indexes of Spanned Samples

In [ ]:
def split_on_sentences_spacy_(text: str) \
        -> typing.List[typing.Tuple[str, int, int]]:
    doc = spacy_model(text)
    sentences = [(sent.text, sent.start_char, sent.end_char)
                 for sent in doc.sents]
    
    return sentences

In [ ]:
def split_on_sentences_natasha_(text: str) \
        -> typing.List[typing.Tuple[str, int, int]]:
    doc = Doc(text)
    doc.segment(natasha_segmenter)
    sentences = [(sent.text, sent.start, sent.stop) for sent in doc.sents]

    return sentences

In [ ]:
def split_on_segments_(text: str, segment_len: int=88, stride: int=88) \
        -> typing.List[typing.Tuple[str, int, int]]:
    segments = [(text[i:i + segment_len], i, i + segment_len - 1)
                for i in range(0, len(text), stride)]

    return segments

In [ ]:
def split_on_sentences(text: str) -> typing.List[typing.Tuple[str, int, int]]:
    match SEGMENTATION_MODEL:
        case 'SPACY':
            return split_on_sentences_spacy_(text)
        case 'NATASHA':
            return split_on_sentences_natasha_(text)
        case 'CUSTOM':
            return split_on_segments_(text)
        case _:
            raise NotImplementedError()

In [ ]:
def get_spanned_sample_idxs(data: typing.List[typing.Dict]) -> typing.List[int]:
    spanned_samples_idxs = []
    for i, sample in enumerate(data):
        ans_start = int(sample['extracted_part']['answer_start'][0])
        ans_stop = int(sample['extracted_part']['answer_end'][0])
        
        for sent, sent_start, sent_stop in data[i]['sents']:
            if (sent_start <= ans_start <= sent_stop
                    and not ans_stop <= sent_stop):
                spanned_samples_idxs.append(i)
    
    return spanned_samples_idxs

In [ ]:
for i in range(len(train)):
    train[i]['sents'] = split_on_sentences(train[i]['text'])

In [ ]:
get_spanned_sample_idxs(train)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 123,
 124,
 126,
 127,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 177,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 198,
 199,
 200,
 201,
 202

#### 2\.2\.2 Segmentation Comparison

In [ ]:
print(train[239]['extracted_part']['text'][0], '\n')

segmenter = Segmenter()
doc = Doc(train[239]['text'])
doc.segment(segmenter)
for sent in doc.sents:
    print(sent.text)

Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере _________________руб. 10 % от цены контракта. 

Контракт № ____ на поставку кроватей механических (ИКЗ 222224200127922420100100140010000244) с. Пещерка «___»_____________2022 г. КГБСУСО «Пещерский психоневрологический интернат», именуемое в дальнейшем «Заказчик», в лице директора Ермоленко Андрея Алексеевича, действующего на основании  Оплата по Контракту осуществляется в безналичном порядке путем перечисления денежных средств со счета Заказчика на счет Поставщика.
Обязательства Заказчика по оплате считаются исполненными с момента списания денежных средств в размере, установленном Контрактом, с лицевого счета Заказчика.
За дальнейшее прохождение денежных средств Заказчик ответственности не несет.
Обязательства Датой оплаты считается дата списания денежных средств со счета Заказчика.
Оплата по Контракту осуществляется после исполнения обязательств Поставщиком по поставке товара.
8

In [ ]:
"""Output sample which spans multiple sentences received
using spacy model."""

print(train[239]['extracted_part']['text'][0], '\n')
for sent in spacy_model(train[239]['text']).sents:
    print(str(sent))

Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере _________________руб. 10 % от цены контракта. 

Контракт № ____ на поставку кроватей механических (ИКЗ 222224200127922420100100140010000244) с. Пещерка «___»_____________2022 г. КГБСУСО «Пещерский психоневрологический интернат», именуемое в дальнейшем «Заказчик», в лице директора Ермоленко Андрея Алексеевича, действующего на основании  Оплата по Контракту осуществляется в безналичном порядке путем перечисления денежных средств со счета Заказчика на счет Поставщика.
Обязательства Заказчика по оплате считаются исполненными с момента списания денежных средств в размере, установленном Контрактом, с лицевого счета Заказчика.
За дальнейшее прохождение денежных средств Заказчик ответственности не несет.
Обязательства Датой оплаты считается дата списания денежных средств со счета Заказчика.
Оплата по Контракту осуществляется после исполнения обязательств Поставщиком по поставке товара.
8

#### 2\.2\.3 Indexes of Target Sentences

In [ ]:
def get_target_sentence_idxs(sample: typing.Dict) -> typing.List[int]:
    ans_start = int(sample['extracted_part']['answer_start'][0])
    ans_stop = int(sample['extracted_part']['answer_end'][0])

    target_sents = []
    for j, (sent, sent_start, sent_stop) in enumerate(sample['sents']):
        if ans_start <= sent_stop and ans_stop >= sent_start:
            target_sents.append(j)
    
    return target_sents

In [ ]:
for i in range(len(train)):
    train[i]['target_sent_idxs'] = get_target_sentence_idxs(train[i])

In [ ]:
for i in range(5):
    print(train[i]['target_sent_idxs'])

[14, 15]
[13, 14]
[14, 15]
[14, 15]
[14, 15]


#### 2\.2\.4 Embeddings of Target sentences

In [ ]:
import numpy as np

def embed_sentence(sent: str) -> typing.List[float]:
    doc = Doc(sent)
    doc.segment(natasha_segmenter)

    vectors = [np.zeros((EMB_DIM,))]
    vectors += [natasha_emb[token.text] for token in doc.tokens
                if token.text in natasha_emb]
    
    return np.asarray(vectors).mean(axis=0)

In [ ]:
target_vectors = {
    'обеспечение исполнения контракта': [],
    'обеспечение гарантийных обязательств': [],
}

for sample in train:
    for sent_idx in sample['target_sent_idxs']:
        vector = embed_sentence(sample['sents'][sent_idx][0])
        target_vectors[sample['label']].append(vector)

for key in target_vectors:
    target_vectors[key] = np.asarray(target_vectors[key]).mean(axis=0)

#### 2\.2\.5 Text Truncation

In [ ]:
from numpy import dot
from numpy.linalg import norm

def truncate_text(sample: typing.Dict) -> str:
    l = 0
    r = len(sample['sents'])

    while l < r:
        # print('l', l)
        sent = sample['sents'][l][0]
        vector = embed_sentence(sent)
        cos_sim = (dot(target_vectors[sample['label']], vector)
            / (norm(target_vectors[sample['label']]) * norm(vector)))
        
        if cos_sim > SIMILARITY_THRESHOLD:
            break

        l += 1

    while l < r:
        # print('r', r)
        sent = sample['sents'][r - 1][0]
        vector = embed_sentence(sent)
        cos_sim = (dot(target_vectors[sample['label']], vector)
            / (norm(target_vectors[sample['label']]) * norm(vector)))
        
        if cos_sim > SIMILARITY_THRESHOLD:
            break

        r -= 1
    
    sents = [sent[0] for sent in sample['sents'][l:r]]
    text = ' '.join(sents)
    return text

In [ ]:
sad = 3

def fun(sample: typing.Dict) -> str:
    coefs = []
    for i, sent in enumerate(list(sample['sents'])):
        vector = embed_sentence(sent[0])
        cos_sim = (dot(target_vectors[sample['label']], vector)
            / (norm(target_vectors[sample['label']]) * norm(vector)))
        
        coefs.append(cos_sim)

    anchor = np.argmax(coefs)
    l = max(0, anchor - sad)
    r = min(anchor + sad + 1, len(coefs))  # Careful here!

    sample['text'] = ''.join([sent[0] for sent in sample['sents'][l:r]])
    
    if sample['extracted_part']['answer_end'][0] == 0:
        return

    
    sample['extracted_part']['answer_start'][0] -= sample['sents'][l][1]
    sample['extracted_part']['answer_end'][0] -= sample['sents'][l][1]

чета в соответствии с п.16 ч. 1 ст. 42 Закона № 44-ФЗ "Номер расчётного счёта" 00000000000000000000 "Номер лицевого счёта" См. прилагаемые документы "БИК" 000000000 "Наименование кредитной организации" "Номер корреспондентского счета" Обеспечение исполнения контракта Требуется обеспечение исполнения контракта Размер обеспечения исполнения контракта 6593.25 Российский рубль Порядок обеспечения исполнения контракта, требования к обеспечению Исполнение контракта, гарантийные обязательства могут обеспечиваться предоставлением независимой гарантии, соответствующей требованиям ст. 45 Федерального закона № 44-ФЗ, ил
Размер обеспечения исполнения контракта 6593.25 Российский рубль


In [ ]:
print(train[775]['target_sent_idxs'])
print(train[775]['text'])
print(train[775]['extracted_part'])

[0]
Акционерное общество «Российский концерн по производству электрической и тепловой энергии на атомных станциях» (АО «Концерн Росэнергоатом») Филиал АО «Концерн Росэнергоатом» «Калининская атомная станция» (Калининская АЭС) г. Удомля, Тверская область, 171841 Телефон (48255) 5-18-64, (48255) 5-43-74 Факс (48255) 5-45-91, e-mail: knpp@knpp.ru ОКПО 08614411, ОГРН 5087746119951 ИНН 7721632827, КПП 691643001 № На № от Открытый запрос предложений ГПЗ № 18981/215 ИЗВЕЩЕНИЕ О ПРОВЕДЕНИИ ЗАКУПКИ 1) Форма и способ процедуры закупки: открытый запрос предложений в электронной форме. 2) Закупка проводится в соответствии с Единым отраслевым стандартом закупок (Положением о закупке) Государственной корпорации по атомной энергии «Росатом» с изменениями, утвержденными решением наблюдательного совета Госкорпорации «Росатом» (протокол от 29.10.2021 № 150), с учетом приказа генерального директора Госкорпорации «Росатом» от 03.08.2021 № 1/981-П, приказа Генерального директора Госкорпорации «Росатом» от 

In [ ]:
for sample in train:
    sample = train[6]
    print('\n', sample['target_sent_idxs'])
    for i, sent in enumerate(list(sample['sents'])):
        vector = embed_sentence(sent[0])
        cos_sim = (dot(target_vectors[sample['label']], vector)
            / (norm(target_vectors[sample['label']]) * norm(vector)))

        # if cos_sim < SIMILARITY_THRESHOLD:

        print(f'{i} {cos_sim:.4f}')
    break


 [14, 15, 16]
0 0.5628
1 0.5590
2 nan
3 0.3805
4 0.5890
5 0.6003
6 0.6481
7 0.7514
8 0.5672
9 0.7620
10 0.5167
11 0.7312
12 0.8204
13 0.6801
14 0.6618
15 0.7952
16 0.7320
17 0.6483
18 0.7170
19 0.8118
20 0.7952
21 0.7301
22 0.7859
23 0.7792
24 0.7750
25 0.7576
26 0.7740
27 0.4147
28 0.5637


<ipython-input-51-2050184a006d>:6: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = (dot(target_vectors[sample['label']], vector)


In [ ]:
for i in range(len(train)):
    train[i]['text_new'] = truncate_text(train[i])

<ipython-input-102-0ccf9f26fdeb>:24: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = (dot(target_vectors[sample['label']], vector)
<ipython-input-102-0ccf9f26fdeb>:12: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = (dot(target_vectors[sample['label']], vector)


In [ ]:
for i in range(len(train)):
    fun(train[i])

<ipython-input-57-44b832ccccc2>:7: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = (dot(target_vectors[sample['label']], vector)


In [ ]:
print(train[307]['text'])
print(train[307]['extracted_part'])

(Идентификационный код закупки № 222450121796445010100100460022790242) г. Курган «____»__________20___г. онтракта не превышает 3 млн. рублей; 5 процентов цены Контракта в случае, если цена Контракта составляет от 3 млн. рублей до 50 млн. рублей (включительно); 1 процент цены Контракта в случае, если цена Контракта составляет от 50 млн. рублей до 100 млн. рублей (включительно). 6.4. Общая сумма начисленных штрафов за неисполнение или ненадлежащее исполнение Поставщиком обязательств, предусмотренных Контрактом, не может превышать цену Контракта. 6.5. Сторона освобождается от уплаты неустойки (штрафа, пени), есл
{'text': ['Обеспечение исполнения Контракта устанавливается в размере 5% от цены Контракта и составляет _____________________ (_____'], 'answer_start': [1105], 'answer_end': [1226]}


In [ ]:
i = 0
for sample in train:
    if len(sample['text_new']) == 0:
        print(i)
    i += 1

775
831
872
877
886
918
924
1035
1045
1046
1048
1080
1115
1130
1157
1220
1418
1460
1470
1471
1492
1521
1523
1526
1534
1537
1566
1573
1599
1605
1607
1617
1622
1638
1643
1653
1672
1676
1678
1688
1694
1697
1705
1708
1713
1749
1790
1793
1798
